In [5]:
import warnings
warnings.filterwarnings('ignore')
#Import general libraries (needed for functions)
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pickle as pk

#Import Qiskit classes classes
import qiskit
from qiskit import IBMQ
from qiskit import Aer

from qiskit.compiler import transpile, assemble
from qiskit.result import marginal_counts as margct
from qiskit.circuit.exceptions import CircuitError
from qiskit.tools.monitor import job_monitor
import qiskit.quantum_info as qi

import mcm_rb_circuits_for_simulation as mcmrb_sims

#aer noise stuff
import qiskit.providers.aer.noise as noise

In [6]:
def save_object(obj,name):
    try:
        with open(name, "wb") as f:
            pk.dump(obj, f, protocol=pk.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

In [7]:
simulator = Aer.get_backend('aer_simulator_density_matrix')

# Error Sweeps

### Error models

In [8]:
# Control qubit errors

# Error probabilities for control qubits
prob_c1 = 0.001  # 1-qubit gate
prob_c2 = 0.01   # 2-qubit gate

# Depolarizing control errors
error_c1 = noise.depolarizing_error(prob_c1, 1)
error_c2 = noise.depolarizing_error(prob_c2, 2)

#T1 and T2 (all same unit (us))
t1 = 345
t2 = 280
meas_time = 0.71
error_t1t2 = noise.thermal_relaxation_error(t1, t2, meas_time).tensor(noise.depolarizing_error(0.0, 1))

In [9]:
T1T2E = noise.thermal_relaxation_error(t1, t2, meas_time).to_quantumchannel()

In [10]:
#Measurement induced Stark shift
def U_stark(phi):
    return sp.linalg.expm(-1j*phi*np.array([[1,0],[0,-1]]))
    
#Probabilistic cross-measurement
def cm_K(cm_prob):
    return [np.sqrt(1-cm_prob)*np.array([[1.0,0.0],[0.0,1.0]]),np.sqrt(cm_prob)*np.array([[1.0,0.0],[0.0,0.0]]),np.sqrt(cm_prob)*np.array([[0.0,0.0],[0.0,1.0]])]


In [11]:
# Combined errors

# T1 T2 + depolarizing
error_t1t2_dep = noise.thermal_relaxation_error(t1, t2, meas_time).tensor(noise.depolarizing_error(prob_a1, 1))

# Collision
Imat = np.array([[1.,0.],[0.,1.]])
Zmat = np.array([[1.,0.],[0.,-1.]])
def U_col(delta,J):
    Ham = J*np.array([[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,0]]) + (delta)*np.kron(Imat,Zmat)/2
    return sp.linalg.expm(-1j*Ham*meas_time)

# ZZ-coupling
Emat = np.array([[0.,0.],[0.,1.]])
ZZ_Ham = np.kron(Zmat,Emat)

def U_ZZ(ZZ_rate):
    return sp.linalg.expm(-1j*ZZ_rate*meas_time*ZZ_Ham)

NameError: name 'prob_a1' is not defined

### Construct circuits

In [14]:
seeds = 60
xval = np.insert(np.ceil(np.geomspace(1,150,14)).astype(int),0,0)
sqrb_list = [0]
ancilla_q_list = [1]

mcm_rb_circs_sim, noise_model_sim = mcmrb_sims.mcm_rb_circs(xval, 
                         seeds=seeds, 
                         rb_list=sqrb_list, 
                         ancilla_list=ancilla_q_list, 
                
                                   make_mcm_only_circuits=True, 
                                   make_delay_circuits=True)


In [29]:
mcm_rb_circs_sim[3].draw()

┌──────────┐ ░ ┌──────────────┐┌─────────┐┌───────────────┐ ░ »
q_0: ┤ Clifford ├─░─┤1             ├┤1        ├┤1              ├─░─»
     └──────────┘ ░ │  pre_meas_id ││  meas_u ││  post_meas_id │ ░ »
q_1: ─────────────░─┤0             ├┤0        ├┤0              ├─░─»
                  ░ └──────────────┘└─────────┘└───────────────┘ ░ »
c: 2/══════════════════════════════════════════════════════════════»
                                                                   »
«     ┌─────────────────────────────────────────────────────────┐ ░ »
«q_0: ┤ Clifford: Stabilizer = ['+Y'], Destabilizer = ['+X']_dg ├─░─»
«     └─────────────────────────────────────────────────────────┘ ░ »
«q_1: ────────────────────────────────────────────────────────────░─»
«                                                                 ░ »
«c: 2/══════════════════════════════════════════════════════════════»
«                                                                   »
«      probabilities ┌─┐   
«q_0: ───────░───────┤M├───
«            ░       └╥┘┌─┐
«q_1: ───────░────────╫─┤M├
«            ░        ║ └╥┘
«c: 2/════════════════╩══╩═
«                     0  1

In [18]:
mcm_rb_tp = transpile(mcm_rb_circs_sim, 
                basis_gates=['id','rz','sx','x','cx','unitary','save_probabilities'], 
                optimization_level=1)

KeyboardInterrupt: 

## Simulate

### non-QND Measurement Error

In [ ]:
# non-QND Error
# Control = Pre-m T1,T2
# Ancilla = Post-m Depolarizing

dep_probs = np.arange(0.02,0.21,0.02)

qnd_sims = []
for x in dep_probs:    
    noise_model = noise.NoiseModel()
    noise_model = noise_model.from_dict(noise_model_sim.to_dict())
    
    noise_model.add_all_qubit_quantum_error(error_c1, ['sx', 'x', 'id'])
    noise_model.add_all_qubit_quantum_error(error_c2, ['cx'])
    
    noise_model.add_quantum_error(error_t1t2, ['pre_meas_id'], [1,0])
    noise_model.add_quantum_error(error_t1t2, ['pre_delay_id'], [1,0])
    
    error_a1 = noise.depolarizing_error(0.0, 1).tensor(noise.depolarizing_error(x, 1))
    noise_model.add_quantum_error(error_a1, ['post_meas_id'], [1,0])
    
    qnd_sims.append(simulator.run(mcm_rb_tp,noise_model=noise_model))

In [ ]:
xvals_qnd = []
rb_res_qnd = []
fits_qnd = []
lfits_qnd = []

for i in range(len(qnd_sims)):
    xvals_temp, rb_res_temp, fits_temp, lfits_temp = mcmrb_sims.fit_mcm_RB_sims(qnd_sims[i], sqrb_list, ancilla_q_list)
    xvals_qnd.append(xvals_temp)
    rb_res_qnd.append(rb_res_temp)
    fits_qnd.append(fits_temp)
    lfits_qnd.append(lfits_temp)

In [ ]:
save_object([xvals_qnd,rb_res_qnd,fits_qnd,lfits_qnd],"qnd_sims1.pickle")

### Stark Shift

In [ ]:
# Control error: Stark
# Control = Pre-m Stark Z Post-m T1,T2
# Ancilla = Ideal

phis = 2*np.pi*20/1000*0.71*np.arange(0.05,1.05,0.05)

stark_sims = []
for x in phis:
    noise_model = noise.NoiseModel()
    noise_model = noise_model.from_dict(noise_model_sim.to_dict())
    
    noise_model.add_all_qubit_quantum_error(error_c1, ['sx', 'x', 'id'])
    noise_model.add_all_qubit_quantum_error(error_c2, ['cx'])
    
    noise_model.add_quantum_error(error_t1t2, ['post_meas_id'], [1,0])
#     noise_model.add_quantum_error(error_t1t2, ['pre_meas_id'], [1,0])
    noise_model.add_quantum_error(error_t1t2, ['post_delay_id'], [1,0])
    
    error_stark = noise.coherent_unitary_error(U_stark(x)).tensor(noise.depolarizing_error(0.0, 1))
    noise_model.add_quantum_error(error_stark, ['pre_meas_id'], [1,0])
#     noise_model.add_quantum_error(error_stark, ['post_meas_id'], [1,0])
    
    stark_sims.append(simulator.run(mcm_rb_tp,shots=shots,noise_model=noise_model))

In [ ]:
xvals_st = []
rb_res_st = []
fits_st = []
lfits_st = []

for i in range(len(stark_sims)):
    xvals_temp, rb_res_temp, fits_temp, lfits_temp = mcmrb_sims.fit_mcm_RB_sims(stark_sims[i], sqrb_list, ancilla_q_list)
    xvals_st.append(xvals_temp)
    rb_res_st.append(rb_res_temp)
    fits_st.append(fits_temp)
    lfits_st.append(lfits_temp)

In [ ]:
save_object([xvals_st,rb_res_st,fits_st,lfits_st],"st_sims1.pickle")

### Cross-Measurement

In [ ]:
# Control error: cross-meas
# Control = Pre-m Prob cross-meas Z Post-m T1,T2
# Ancilla = Ideal

cm_probs = np.arange(0.01,0.21,0.01)

cm_sims = []
for x in cm_probs:
    noise_model = noise.NoiseModel()
    noise_model = noise_model.from_dict(noise_model_sim.to_dict())
    
    noise_model.add_all_qubit_quantum_error(error_c1, ['sx', 'x', 'id'])
    noise_model.add_all_qubit_quantum_error(error_c2, ['cx'])
    
    noise_model.add_quantum_error(error_t1t2, ['post_meas_id'], [1,0])
    noise_model.add_quantum_error(error_t1t2, ['post_delay_id'], [1,0])
    
    error_cm = noise.kraus_error(cm_K(x)).tensor(noise.depolarizing_error(0.0, 1))
    noise_model.add_quantum_error(error_cm, ['pre_meas_id'], [1,0])
    
    cm_sims.append(simulator.run(mcm_rb_tp,shots=shots,noise_model=noise_model))

In [ ]:
xvals_cm = []
rb_res_cm = []
fits_cm = []
lfits_cm = []

for i in range(len(cm_sims)):
    xvals_temp, rb_res_temp, fits_temp, lfits_temp = mcmrb_sims.fit_mcm_RB_sims(cm_sims[i], sqrb_list, ancilla_q_list)
    xvals_cm.append(xvals_temp)
    rb_res_cm.append(rb_res_temp)
    fits_cm.append(fits_temp)
    lfits_cm.append(lfits_temp)

In [ ]:
save_object([xvals_cm,rb_res_cm,fits_cm,lfits_cm],"cm_sims1.pickle")

### Collision

In [ ]:
# Collision
# Control = Pre-m Collision Z Post-m T1,T2
# Ancilla = Ideal

deltas = 2*np.pi*np.array([0.0,1.0,2.0,5.0,10.0,20.0,50.0,100.0,200.0,500.0])

col_sims = []
for x in deltas:
    noise_model = noise.NoiseModel()
    noise_model = noise_model.from_dict(noise_model_sim.to_dict())
    
    noise_model.add_all_qubit_quantum_error(error_c1, ['sx', 'x', 'id'])
    noise_model.add_all_qubit_quantum_error(error_c2, ['cx'])
    
    noise_model.add_quantum_error(error_t1t2, ['post_meas_id'], [1,0])
    noise_model.add_quantum_error(error_t1t2, ['post_delay_id'], [1,0])
    
    error_col= noise.coherent_unitary_error(U_col(x,2*np.pi))
    noise_model.add_quantum_error(error_col, ['pre_meas_id'], [1,0])
    
    col_sims.append(simulator.run(mcm_rb_tp,shots=shots,noise_model=noise_model))

In [ ]:
xvals_col = []
rb_res_col = []
fits_col = []
lfits_col = []

for i in range(len(col_sims)):
    xvals_temp, rb_res_temp, fits_temp, lfits_temp = mcmrb_sims.fit_mcm_RB_sims(col_sims[i], sqrb_list, ancilla_q_list)
    xvals_col.append(xvals_temp)
    rb_res_col.append(rb_res_temp)
    fits_col.append(fits_temp)
    lfits_col.append(lfits_temp)

In [ ]:
save_object([xvals_col,rb_res_col,fits_col,lfits_col],"col_sims1.pickle")

### ZZ-coupling

In [ ]:
# ZZ
# Control = Pre-m ZZ Z Post-m T1,T2
# Ancilla = Pre-m ZZ Post-m T1,T2

ZZ0 = 2*np.pi*50.0/1000
T1s = [100.0,10.0,1.0,0.1,0.01]

ZZ_sims = []
for x in T1s:
    
    error_t1t2_both = noise.thermal_relaxation_error(t1, t2, meas_time).tensor(noise.thermal_relaxation_error(x, x/3, meas_time))
    noise_model = noise.NoiseModel()
    noise_model = noise_model.from_dict(noise_model_sim.to_dict())
    
    noise_model.add_all_qubit_quantum_error(error_c1, ['sx', 'x', 'id'])
    
    noise_model.add_quantum_error(error_t1t2_both, ['post_meas_id'], [1,0])
    noise_model.add_quantum_error(error_t1t2_both, ['post_delay_id'], [1,0])
    
    error_ZZ = noise.coherent_unitary_error(U_ZZ(ZZ0))
    noise_model.add_quantum_error(error_ZZ, ['pre_meas_id'], [1,0])
    
    ZZ_sims.append(simulator.run(mcm_rb_tp,noise_model=noise_model))

In [ ]:
xvals_ZZ = []
rb_res_ZZ = []
fits_ZZ = []
lfits_ZZ = []

for i in range(len(ZZ_sims)):
    xvals_temp, rb_res_temp, fits_temp, lfits_temp = mcmrb_sims.fit_mcm_RB_sims_free(ZZ_sims2[i], sqrb_list, ancilla_q_list)
    xvals_ZZ.append(xvals_temp)
    rb_res_ZZ.append(rb_res_temp)
    fits_ZZ.append(fits_temp)
    lfits_ZZ.append(lfits_temp)

In [ ]:
save_object([xvals_ZZ,rb_res_ZZ,fits_ZZ,lfits_ZZ],"ZZ_sims1.pickle")